In [36]:
%load_ext autoreload
%autoreload 2

import yaml
import mesa
from mesa.experimental import JupyterViz
import solara
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from trans_infra.trans_infra.simulator import TransInfraNetworkModel
import networkx as nx
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
def make_trans_graph(model):
    G_trans = model.G_trans
    fig = Figure()
    ax = fig.subplots()
    
    osmid_2_idx = {k: v for (k, v) in zip(G_trans.nodes, range(len(G_trans.nodes))) }
    
    nodes, edges = G_trans.nodes(data=True), G_trans.edges(data=True)
    
    max_road_len = max([ d['length'] for (u, v, d) in edges ])
    layout = { n[0] : [ n[1]['x'], n[1]['y'] ] for n in G_trans.nodes(data=True) }
    
    node_color = [ v['color'] for (k, v) in nodes ]
    edge_color = [ d['color'] for (u, v, d) in edges ]      
    edge_weight = [ d['length'] / max_road_len for (u, v, d) in edges ]
    node_weight = [1] * len(nodes)
    for a in model.space.get_all_cell_contents():
        node_weight[ osmid_2_idx[a.pos] ] += 3

    nx.draw_networkx(G_trans, pos=layout,
                    edge_color=edge_color, 
                    node_color=node_color,
                    width=edge_weight,
                    node_size=node_weight,
                    with_labels=False,
                    ax=ax)
    
    ax.set_title("Places and Their Connections")
    solara.FigureMatplotlib(fig)

In [38]:
def make_social_graph(model):
    fig = Figure()
    ax = fig.subplots()
    
    G_socia = nx.from_scipy_sparse_array(model.A_social)
    pos = nx.spring_layout(G_socia)
    betCent = nx.betweenness_centrality(G_socia, normalized=True, endpoints=True)
    node_color = [20000.0 * G_socia.degree(v) for v in G_socia]
    node_size =  [v * 1000 for v in betCent.values()]
    nx.draw_networkx(G_socia, pos=pos, with_labels=True,
                    node_color=node_color, node_size=node_size,
                    width=[e[2] for e in G_socia.edges.data('weight')],
                    font_size=5, ax=ax)
    
    ax.set_title("Social Ties")
    solara.FigureMatplotlib(fig)

In [39]:
def make_resource_graph(model):
    fig = Figure()
    ax = fig.subplots()
    
    data = []
    for a in model.space.get_all_cell_contents():
        data.append([a.unique_id, *a.curr_resources])
    df = pd.DataFrame(data, columns=["Agent ID", "Sleep", "Work", "Social"])

    ax.bar(["Sleep", "Work", "Social"],
           df[["Sleep", "Work", "Social"]].mean(axis=0).to_list(),
           yerr=df[["Sleep", "Work", "Social"]].std(axis=0).to_list(),
           ecolor='black', capsize=10, alpha=0.5)
    ax.set_title("Agent Resource Counts")

    solara.FigureMatplotlib(fig)
    

In [40]:
def agent_portrayal(graph):
    nodes, edges = graph.nodes(data=True), graph.edges(data=True)
    
    max_road_len = max([ d['length'] for (u, v, d) in edges ])

    node_color = [ v['color'] for (k, v) in nodes ]
    edge_color = [ d['color'] for (u, v, d) in edges ]      
    edge_weight = [ d['length'] / max_road_len for (u, v, d) in edges ]
    
    return {
        "width": edge_weight,
        "edge_color": edge_color,
        "node_color": node_color,
        "node_size": 10
    }

In [41]:
import networkx as nx
import osmnx as ox

def load_network(graph_file) -> nx.Graph:
    """loads in modified OSM graph"""
    G_trans = ox.load_graphml(
                graph_file,
                node_dtypes={'idx':int, 'x':float, 'y':float, 'general0':float, 'general1':float, 
                            'general2':float, 'general3':float, 'general4':float},    
                edge_dtypes={'u':int, 'v':int, 'speed':float, 'capacity':float, 'length':float,
                             'general0':float, 'general1':float, 'general2':float, 'general3':float})
    G_trans = G_trans.to_undirected()                                   # make undirected
    return G_trans

In [46]:
osm_file = "./osm_dataset/raw/bradenton.osm"
graph = load_network(osm_file)

model_params = {
    "num_agents": {
        "type": "SliderInt",
        "value": 100,
        "label": "Number of agents:",
        "min": 25,
        "max": 300,
        "step": 1,
    },
    "max_episode": {
        "type": "SliderInt",
        "value": 24,
        "label": "Number of agents:",
        "min": 10,
        "max": 24,
        "step": 1,
    },
    "graph": graph
}

In [47]:
page = JupyterViz(
    TransInfraNetworkModel,
    model_params,
    measures=[make_trans_graph, make_social_graph, make_resource_graph],
    name="Transit Network Model",
    agent_portrayal=agent_portrayal,
)
# This is required to render the visualization in the Jupyter notebook
page

Cannot show ipywidgets in text